In [2]:
!pip install torch
!pip install datasets
# this notebook use the datasets from the torchdrug

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00


In [3]:
!git clone https://github.com/DeepGraphLearning/torchdrug
!pip install torchdrug
!pip install torch-scatter
!pip install torch-cluster


Cloning into 'torchdrug'...
remote: Enumerating objects: 1381, done.
remote: Counting objects: 100% (752/752), done.
remote: Compressing objects: 100% (327/327), done.
remote: Total 1381 (delta 473), reused 597 (delta 404), pack-reused 629
Receiving objects: 100% (1381/1381), 2.65 MiB | 12.91 MiB/s, done.
Resolving deltas: 100% (739/739), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 13.7 MB/s eta 0:00:0

In [4]:
from google.colab import drive

# Mount Google Drive to access files
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Check current permissions
!ls -l /content/torchdrug/torchdrug

# Add read permission if necessary
!chmod +r /content/torchdrug/torchdrug

import os

# Specify the directory path you want to change to
new_directory = "/content/torchdrug/torchdrug"

# Change the current working directory to the specified path
os.chdir(new_directory)

# Verify the current working directory has changed
current_directory = os.getcwd()
print(f"Current working directory: {current_directory}")

total 48
drwxr-xr-x 2 root root 4096 Nov 21 17:35 core
drwxr-xr-x 3 root root 4096 Nov 21 17:35 data
drwxr-xr-x 2 root root 4096 Nov 21 17:35 datasets
-rw-r--r-- 1 root root  331 Nov 21 17:35 __init__.py
drwxr-xr-x 4 root root 4096 Nov 21 17:35 layers
drwxr-xr-x 3 root root 4096 Nov 21 17:35 metrics
drwxr-xr-x 2 root root 4096 Nov 21 17:35 models
-rw-r--r-- 1 root root 5625 Nov 21 17:35 patch.py
drwxr-xr-x 2 root root 4096 Nov 21 17:35 tasks
drwxr-xr-x 2 root root 4096 Nov 21 17:35 transforms
drwxr-xr-x 4 root root 4096 Nov 21 17:35 utils
Current working directory: /content/torchdrug/torchdrug


In [6]:
# This part has been modifed to use the ESM model to predict the PP interaction rather than CNN model
# there are two versions of ems model, esm-1b and esm-1v. this code uses esm-1b.
from torchdrug import models

# Specify the path to the ESM model weights file in Google Drive
model_weights_path = "/content/drive/MyDrive/model"  # Replace with the your path

# Create an instance of the ESM model, this code is directly from the website
model = models.ESM(path=model_weights_path, model='ESM-1b', readout='mean')



In [7]:
# This part uses the residue values to trasnform the protein
from torchdrug import transforms

truncate_transform = transforms.TruncateProtein(max_length=200, random=False)
protein_view_transform = transforms.ProteinView(view="residue")
transform = transforms.Compose([truncate_transform, protein_view_transform])


In [8]:
#imports data of BetaLactamase from the torchdurg. We can replace this part without own datasets
from torchdrug import datasets

dataset = datasets.BetaLactamase("~/protein-datasets/", atom_feature=None, bond_feature=None, residue_feature="default", transform=transform)
train_set, valid_set, test_set = dataset.split()
print("The label of first sample: ", dataset[0][dataset.target_fields[0]])
print("train samples: %d, valid samples: %d, test samples: %d" % (len(train_set), len(valid_set), len(test_set)))


INFO:torchdrug.utils.file:Downloading https://miladeepgraphlearningproteindata.s3.us-east-2.amazonaws.com/peerdata/beta_lactamase.tar.gz to /root/protein-datasets/beta_lactamase.tar.gz


18:22:14   Downloading https://miladeepgraphlearningproteindata.s3.us-east-2.amazonaws.com/peerdata/beta_lactamase.tar.gz to /root/protein-datasets/beta_lactamase.tar.gz


INFO:torchdrug.utils.file:Extracting /root/protein-datasets/beta_lactamase.tar.gz to /root/protein-datasets


18:22:15   Extracting /root/protein-datasets/beta_lactamase.tar.gz to /root/protein-datasets


Constructing proteins from sequences: 100%|██████████| 5198/5198 [00:15<00:00, 339.87it/s]

The label of first sample:  0.9426838159561157
train samples: 4158, valid samples: 520, test samples: 520


In [9]:
# We first build the dataset via datasets.PPIAffinity, in which each sample is a pair of proteins,
# and it is associated with a continuous label indicating the binding affinity.
# Since we now need to perform transformation on both proteins, we need to specify keys in the transformation function.
# This code only utilize the residue values. Atom featurs, bond futures are omitted.
truncate_transform_ = transforms.TruncateProtein(max_length=200, keys=("graph1", "graph2"))
protein_view_transform_ = transforms.ProteinView(view="residue", keys=("graph1", "graph2"))
transform_ = transforms.Compose([truncate_transform_, protein_view_transform_])
dataset = datasets.PPIAffinity("~/protein-datasets/", atom_feature=None, bond_feature=None, residue_feature="default", transform=transform_)
train_set, valid_set, test_set = dataset.split()
print("The label of first sample: ", dataset[0][dataset.target_fields[0]])
print("train samples: %d, valid samples: %d, test samples: %d" % (len(train_set), len(valid_set), len(test_set)))


INFO:torchdrug.utils.file:Downloading https://miladeepgraphlearningproteindata.s3.us-east-2.amazonaws.com/ppidata/ppi_affinity.zip to /root/protein-datasets/ppi_affinity.zip


18:22:30   Downloading https://miladeepgraphlearningproteindata.s3.us-east-2.amazonaws.com/ppidata/ppi_affinity.zip to /root/protein-datasets/ppi_affinity.zip


INFO:torchdrug.utils.file:Extracting /root/protein-datasets/ppi_affinity.zip to /root/protein-datasets


18:22:31   Extracting /root/protein-datasets/ppi_affinity.zip to /root/protein-datasets


Constructing proteins from sequences: 100%|██████████| 2950/2950 [00:10<00:00, 273.91it/s]

The label of first sample:  -12.2937
train samples: 2421, valid samples: 203, test samples: 326


In [10]:
# To perform PPI affinity prediction, we wrap the ESM encoder into the tasks.InteractionPrediction module
# which appends a task-specific MLP prediction head upon ESM model.
from torchdrug import tasks

task = tasks.PropertyPrediction(model, task=dataset.tasks,
                                criterion="mse", metric=("mae", "rmse", "spearmanr"),
                                normalization=False, num_mlp_layer=2)


In [12]:
# Empties the cuda to reduce the storage
torch.cuda.empty_cache()

In [13]:
# This part is added to specify the split size to reduce the memory. can be eliminated.
# If eliminated, all datasets will be utilized.
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'


In [14]:
# model training process. Modify batch size. Will take a long time. Running aborted if you see some results.
import torch
from torchdrug import core

optimizer = torch.optim.Adam(task.parameters(), lr=1e-4)
solver = core.Engine(task, train_set, valid_set, test_set, optimizer,
                     gpus=[0], batch_size=1)
solver.train(num_epoch=10)
solver.evaluate("valid")


18:22:44   Preprocess training set


 'class': 'core.Engine',
 'gpus': [0],
 'gradient_interval': 1,
 'log_interval': 100,
 'logger': 'logging',
 'num_worker': 0,
 'optimizer': {'amsgrad': False,
               'betas': (0.9, 0.999),
               'capturable': False,
               'class': 'optim.Adam',
               'differentiable': False,
               'eps': 1e-08,
               'foreach': None,
               'fused': None,
               'lr': 0.0001,
               'maximize': False,
               'weight_decay': 0},
 'scheduler': None,
 'task': {'class': 'tasks.InteractionPrediction',
          'criterion': 'mse',
          'graph_construction_model': None,
          'metric': ('mae', 'rmse', 'spearmanr'),
          'mlp_batch_norm': False,
          'mlp_dropout': 0,
          'model': {'class': 'models.ESM',
                    'model': 'ESM-1b',
                    'path': '/content/drive/MyDrive/model',
                    'readout': 'mean'},
          'normalization': False,
          'num_class': None

18:22:54   {'batch_size': 1,
 'class': 'core.Engine',
 'gpus': [0],
 'gradient_interval': 1,
 'log_interval': 100,
 'logger': 'logging',
 'num_worker': 0,
 'optimizer': {'amsgrad': False,
               'betas': (0.9, 0.999),
               'capturable': False,
               'class': 'optim.Adam',
               'differentiable': False,
               'eps': 1e-08,
               'foreach': None,
               'fused': None,
               'lr': 0.0001,
               'maximize': False,
               'weight_decay': 0},
 'scheduler': None,
 'task': {'class': 'tasks.InteractionPrediction',
          'criterion': 'mse',
          'graph_construction_model': None,
          'metric': ('mae', 'rmse', 'spearmanr'),
          'mlp_batch_norm': False,
          'mlp_dropout': 0,
          'model': {'class': 'models.ESM',
                    'model': 'ESM-1b',
                    'path': '/content/drive/MyDrive/model',
                    'readout': 'mean'},
          'normalization': False

18:22:54   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


18:22:54   Epoch 0 begin


...

18:56:45   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


18:56:45   mean squared error: 14.8076


18:57:52   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


18:57:52   mean squared error: 15.9798


KeyboardInterrupt: ignored